# Úvod do prompt engineeringu
Prompt engineering je proces navrhování a optimalizace promptů pro úlohy zpracování přirozeného jazyka. Zahrnuje výběr vhodných promptů, ladění jejich parametrů a hodnocení jejich výkonu. Prompt engineering je klíčový pro dosažení vysoké přesnosti a efektivity NLP modelů. V této části se seznámíme se základy prompt engineeringu s využitím modelů OpenAI pro zkoumání.


### Cvičení 1: Tokenizace
Prozkoumejte tokenizaci pomocí tiktoken, rychlého open-source tokenizéru od OpenAI.
Další příklady najdete v [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst).


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### Cvičení 2: Ověření nastavení klíče Github Models

Spusťte níže uvedený kód a ověřte, že váš endpoint Github Models je správně nastavený. Kód pouze zkouší jednoduchý základní prompt a ověřuje výsledek. Vstup `oh say can you see` by měl být doplněn například jako `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### Cvičení 3: Výmysly
Prozkoumejte, co se stane, když požádáte LLM, aby vygeneroval odpověď na dotaz týkající se tématu, které možná neexistuje, nebo o tématech, o kterých nemusí vědět, protože byly mimo jeho předtrénovanou sadu dat (například novější události). Sledujte, jak se odpověď změní, pokud zkusíte jiný dotaz nebo jiný model.


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### Cvičení 4: Na základě instrukce
Použijte proměnnou "text" k nastavení hlavního obsahu 
a proměnnou "prompt" k zadání instrukce související s tímto hlavním obsahem.

Zde žádáme model, aby shrnul text pro žáka druhé třídy.


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### Cvičení 5: Složitý prompt
Vyzkoušejte požadavek, který obsahuje systémové, uživatelské a asistentské zprávy  
Systém nastavuje kontext asistenta  
Uživatelské a asistentské zprávy poskytují kontext pro vícekrokovou konverzaci

Všimněte si, jak je osobnost asistenta nastavena na „sarkastickou“ v systémovém kontextu.  
Zkuste použít jiný kontext osobnosti. Nebo zkuste jinou sérii vstupních/výstupních zpráv


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### Cvičení: Prozkoumejte svou intuici
Výše uvedené příklady vám ukazují vzory, které můžete použít k vytváření nových promptů (jednoduchých, složitých, instrukčních atd.) – zkuste vytvořit další cvičení, abyste prozkoumali některé další nápady, o kterých jsme mluvili, například příklady, nápovědy a další.



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro kritické informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
